In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import DataLoader, Machine_Model, Engine, Information
import torch
import optuna

# seed = 45
# pres = [0.0246, 0.025, 0.0254, 0.0271, 0.0279, 0.0396, 0.0404] # [0.0279, 0.0271, 0.0404, 0.0396, 0.025]
train_percent = 0.7
val_percent = 0.3
test_percent = None
batch_size = 100
# torch.manual_seed(seed)
data_path = "data/data_complete.csv"

def objective(trial):
    INPUT_SIZE = 1
    HIDDEN_UNITS = trial.suggest_int("hidden_units", 10, 50)
    OUTPUT_SIZE = 1
    DEPTH = trial.suggest_int('depth', 1, 5)

    dataloader = DataLoader.MyDataloader(file_path=data_path, train_percent=train_percent, val_percent=val_percent,
                                        test_percent=test_percent, batch_size=batch_size)
    train, val, test = dataloader.fit()

    try:
        print(len(train), len(val), len(test))
    except:
        print(len(train), len(val))

    # INPUT_SIZE = len(Information.features)

    # torch.manual_seed(seed)
    model0 = Machine_Model.LSTM_V0(input_size=INPUT_SIZE, hidden_size=HIDDEN_UNITS, output_size=3, depth_number=DEPTH)
    # torch.manual_seed(seed)
    model1 = Machine_Model.NN_V0(input_size=17, hidden_size=HIDDEN_UNITS, output_size=OUTPUT_SIZE)

    # EPOCHS_list = [1382, 1423]
    # EPOCHS = EPOCHS_list[1]
    EPOCHS = 1500
    loss_fn = torch.nn.MSELoss()
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-1, log=True)
    optimizer = torch.optim.Adam(model0.parameters(), learning_rate)
    optimizer1 = torch.optim.Adam(model1.parameters(), learning_rate, weight_decay=weight_decay)

    # torch.manual_seed(seed)
    engine = Engine.Machine_Engine(model=model0, model1=model1, train_dataloader=train, val_dataloader=val,
                                test_dataloader=test)
    # torch.manual_seed(seed)
    model0_results = engine.train(loss_fn=loss_fn, optimizer=optimizer, optimizer1=optimizer1, epochs_num=EPOCHS,
                                early_stop_patience=None, writer=False, resolution=1)
    
    return model0_results["val_loss"]

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

print("Best hyperparameters: ", study.best_params)


[I 2024-05-19 23:54:13,079] A new study created in memory with name: no-name-219e95e4-484c-4477-a065-02a40267be10


1 1


  0%|          | 0/1500 [00:00<?, ?it/s]

Epoch 1 | train: Loss 0.616300 Accuracy -21.6018 | validation: Loss 2.008386 Accuracy -125.1727
Epoch 2 | train: Loss 0.292755 Accuracy -9.7363 | validation: Loss 0.118164 Accuracy -6.4234
Epoch 3 | train: Loss 0.049796 Accuracy -0.8262 | validation: Loss 0.073399 Accuracy -3.6112
Epoch 4 | train: Loss 0.064498 Accuracy -1.3654 | validation: Loss 0.061511 Accuracy -2.8643
Epoch 5 | train: Loss 0.042314 Accuracy -0.5518 | validation: Loss 0.064972 Accuracy -3.0817
Epoch 6 | train: Loss 0.023661 Accuracy 0.1323 | validation: Loss 0.090944 Accuracy -4.7134
Epoch 7 | train: Loss 0.026356 Accuracy 0.0334 | validation: Loss 0.089937 Accuracy -4.6501
Epoch 8 | train: Loss 0.027590 Accuracy -0.0118 | validation: Loss 0.061180 Accuracy -2.8435
Epoch 9 | train: Loss 0.021193 Accuracy 0.2228 | validation: Loss 0.041817 Accuracy -1.6271
Epoch 10 | train: Loss 0.019919 Accuracy 0.2695 | validation: Loss 0.037318 Accuracy -1.3444
Epoch 11 | train: Loss 0.022379 Accuracy 0.1793 | validation: Loss 0.0

[W 2024-05-19 23:54:25,984] Trial 0 failed with parameters: {'hidden_units': 15, 'depth': 1, 'learning_rate': 0.09954748354475935, 'weight_decay': 0.012988817632185535} because of the following error: The number of the values 1500 did not match the number of the objectives 1.
[W 2024-05-19 23:54:25,993] Trial 0 failed with value [2.0083858966827393, 0.11816372722387314, 0.07339923083782196, 0.061511073261499405, 0.06497222930192947, 0.09094373881816864, 0.08993729948997498, 0.06118002533912659, 0.04181719571352005, 0.037317998707294464, 0.03344135731458664, 0.06185782700777054, 0.039735324680805206, 0.0307464599609375, 0.030562778934836388, 0.031170129776000977, 0.05972468852996826, 0.06176064908504486, 0.04324307292699814, 0.0787721499800682, 0.06770604103803635, 0.025680219754576683, 0.029057465493679047, 0.030216967687010765, 0.032131895422935486, 0.027336319908499718, 0.05486733838915825, 0.02685409225523472, 0.025757886469364166, 0.05913310497999191, 0.059724293649196625, 0.023775

...

  0%|          | 0/1500 [00:00<?, ?it/s]

Epoch 1 | train: Loss 0.108212 Accuracy -2.9685 | validation: Loss 0.149280 Accuracy -8.3782
Epoch 2 | train: Loss 0.091772 Accuracy -2.3656 | validation: Loss 0.113598 Accuracy -6.1365
Epoch 3 | train: Loss 0.077669 Accuracy -1.8484 | validation: Loss 0.084734 Accuracy -4.3232
Epoch 4 | train: Loss 0.065718 Accuracy -1.4101 | validation: Loss 0.061977 Accuracy -2.8936
Epoch 5 | train: Loss 0.055802 Accuracy -1.0465 | validation: Loss 0.045291 Accuracy -1.8453
Epoch 6 | train: Loss 0.047781 Accuracy -0.7523 | validation: Loss 0.033116 Accuracy -1.0804
Epoch 7 | train: Loss 0.041467 Accuracy -0.5207 | validation: Loss 0.024873 Accuracy -0.5626
Epoch 8 | train: Loss 0.036741 Accuracy -0.3474 | validation: Loss 0.020084 Accuracy -0.2617
Epoch 9 | train: Loss 0.033397 Accuracy -0.2248 | validation: Loss 0.018273 Accuracy -0.1480
Epoch 10 | train: Loss 0.031221 Accuracy -0.1450 | validation: Loss 0.019262 Accuracy -0.2101
Epoch 11 | train: Loss 0.029950 Accuracy -0.0984 | validation: Loss 0

[W 2024-05-19 23:55:20,820] Trial 4 failed with parameters: {'hidden_units': 11, 'depth': 1, 'learning_rate': 0.0022170431245120644, 'weight_decay': 0.05906297894712533} because of the following error: The number of the values 1500 did not match the number of the objectives 1.
[W 2024-05-19 23:55:20,820] Trial 4 failed with value [0.14928029477596283, 0.11359772086143494, 0.08473378419876099, 0.06197739392518997, 0.04529114067554474, 0.03311576321721077, 0.02487329952418804, 0.020083801820874214, 0.018273206427693367, 0.019262051209807396, 0.02217411994934082, 0.026198256760835648, 0.030574439093470573, 0.03465098515152931, 0.03793857619166374, 0.04013620689511299, 0.04112866520881653, 0.04096784070134163, 0.039826661348342896, 0.037950366735458374, 0.03547980636358261, 0.03264990448951721, 0.02976657636463642, 0.027134981006383896, 0.0249557476490736, 0.02328871376812458, 0.022098533809185028, 0.021283846348524094, 0.020862506702542305, 0.02072637900710106, 0.02078433893620968, 0.0208

Epoch 1489 | train: Loss 0.003573 Accuracy 0.8690 | validation: Loss 0.040171 Accuracy -1.5237
Epoch 1490 | train: Loss 0.003548 Accuracy 0.8699 | validation: Loss 0.039924 Accuracy -1.5082
Epoch 1491 | train: Loss 0.003753 Accuracy 0.8623 | validation: Loss 0.039870 Accuracy -1.5047
Epoch 1492 | train: Loss 0.003367 Accuracy 0.8765 | validation: Loss 0.040023 Accuracy -1.5144
Epoch 1493 | train: Loss 0.003586 Accuracy 0.8685 | validation: Loss 0.039934 Accuracy -1.5088
Epoch 1494 | train: Loss 0.003489 Accuracy 0.8720 | validation: Loss 0.039795 Accuracy -1.5000
Epoch 1495 | train: Loss 0.003369 Accuracy 0.8764 | validation: Loss 0.039677 Accuracy -1.4926
Epoch 1496 | train: Loss 0.003524 Accuracy 0.8708 | validation: Loss 0.039599 Accuracy -1.4877
Epoch 1497 | train: Loss 0.003331 Accuracy 0.8779 | validation: Loss 0.039796 Accuracy -1.5001
Epoch 1498 | train: Loss 0.003446 Accuracy 0.8736 | validation: Loss 0.040080 Accuracy -1.5179
Epoch 1499 | train: Loss 0.003442 Accuracy 0.8738 

ValueError: No trials are completed yet.